In [1]:
from sympy import *
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
t = symbols('t')
#states
x = Function('x')(t)
y = Function('y')(t)
th1 = Function('θ_1')(t)
th2 = Function('θ_2')(t)
all_states = [x,y,th1,th2]

#parameters
r1, rj, r2 = symbols('r1, r_j, r2')
m1, m2, I1, I2, g = symbols('m1, m2, I1, I2, g')

#Visuals
#use clean() to render d/dt as dots and remove the (t) in functions
dots = {}
no_t = {}
for state in all_states:
    d_disp = Function('\dot{' + state.name + '}')(t)
    dd_disp = Function('\ddot{' + state.name + '}')(t)
    dots[state.diff(t)] = d_disp
    dots[d_disp.diff(t)] = dd_disp
    no_t[state] = symbols(state.name)
    no_t[d_disp] = symbols('\dot{' + state.name + '}')
    no_t[dd_disp] = symbols('\ddot{' + state.name + '}')
clean = lambda expr : expr.subs(dots).subs(dots).subs(no_t)
half = Rational(1,2)    #keep as fraction rather than floating point

clean(th1.diff(t))

\dot{θ_1}

In [3]:
'''Kinematics'''
x1 = -r1*sin(th1)
y1 = r1*cos(th1)
xj = -rj*sin(th1)
yj = rj*cos(th1)
x2 = xj + r2*sin(th1 + th2)
y2 = yj - r2*cos(th1 + th2)

display(
    clean(Eq(MatrixSymbol('P1', 2, 1), Matrix([x1,y1]))),
    clean(Eq(MatrixSymbol('P_j', 2, 1), Matrix([xj,yj]))),
    clean(Eq(MatrixSymbol('P2', 2, 1), Matrix([x2,y2]))),
)

Eq(P1, Matrix([
[-r1*sin(θ_1)],
[ r1*cos(θ_1)]]))

Eq(P_j, Matrix([
[-r_j*sin(θ_1)],
[ r_j*cos(θ_1)]]))

Eq(P2, Matrix([
[ r2*sin(θ_1 + θ_2) - r_j*sin(θ_1)],
[-r2*cos(θ_1 + θ_2) + r_j*cos(θ_1)]]))

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

plt.ion()

def plot_linkage(th1, th2, r1, r2, rj):
    # Convert angles to radians
    th1_rad = np.radians(th1)
    th2_rad = np.radians(th2)
    
    # Calculate points
    x1 = -r1 * np.sin(th1_rad)
    y1 = r1 * np.cos(th1_rad)
    xj = -rj * np.sin(th1_rad)
    yj = rj * np.cos(th1_rad)
    x2 = xj + r2 * np.sin(th1_rad + th2_rad)
    y2 = yj - r2 * np.cos(th1_rad + th2_rad)
    
    # Create plot
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Plot origin point
    ax.plot(0, 0, 'ko', markersize=10, label='Origin')
    
    # Plot first link (origin to joint)
    ax.plot([0, xj], [0, yj], 'b-', linewidth=2, label='Link 1')
    
    # Plot second link (joint to end)
    ax.plot([xj, x2], [yj, y2], 'r-', linewidth=2, label='Link 2')
    
    # Plot point P1
    ax.plot(x1, y1, 'go', markersize=8, label='P1')
    
    # Plot joint point
    ax.plot(xj, yj, 'mo', markersize=8, label='Joint')
    
    # Plot end point
    ax.plot(x2, y2, 'ko', markersize=8, label='P2')
    
    # Set equal aspect ratio and limits
    ax.set_aspect('equal')
    ax.set_xlim(-0.2, 0.2)
    ax.set_ylim(-0.05, 0.2)
    
    # Add grid and legend
    ax.grid(True)
    ax.legend()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('2D Linkage Visualization')
    
    # plt.close()
    return fig

# Create interactive widget
interactive_plot = interactive(
    plot_linkage,
    th1=widgets.FloatSlider(min=-180, max=180, step=1, value=30, description='θ1 (deg)'),
    th2=widgets.FloatSlider(min=-180, max=180, step=1, value=30, description='θ2 (deg)'),
    r1=widgets.FloatSlider(min=0.01, max=0.20, step=0.01, value=0.06, description='r1'),
    r2=widgets.FloatSlider(min=0.01, max=0.20, step=0.01, value=0.06, description='r2'),
    rj=widgets.FloatSlider(min=0.01, max=0.20, step=0.01, value=0.10, description='rj')
)

# Display the interactive plot
display(interactive_plot)

interactive(children=(FloatSlider(value=30.0, description='θ1 (deg)', max=180.0, min=-180.0, step=1.0), FloatS…

In [5]:
KE_lin = half*m1*(x1.diff(t)**2 + y1.diff(t)**2) + half*m2*(x2.diff(t)**2 + y2.diff(t)**2)
KE_ang = half*I1*(th1.diff(t)**2) + half*I2*((th1 + th2).diff(t)**2)
KE = simplify(KE_lin + KE_ang)

PE = m1*g*y1 + m2*g*y2
L = simplify(KE - PE)

display(clean(KE), clean(PE), clean(L))

I1*\dot{θ_1}**2/2 + I2*(\dot{θ_1} + \dot{θ_2})**2/2 + \dot{θ_1}**2*m1*r1**2/2 + m2*(\dot{θ_1}**2*r2**2 - 2*\dot{θ_1}**2*r2*r_j*cos(θ_2) + \dot{θ_1}**2*r_j**2 + 2*\dot{θ_1}*\dot{θ_2}*r2**2 - 2*\dot{θ_1}*\dot{θ_2}*r2*r_j*cos(θ_2) + \dot{θ_2}**2*r2**2)/2

g*m1*r1*cos(θ_1) + g*m2*(-r2*cos(θ_1 + θ_2) + r_j*cos(θ_1))

I1*\dot{θ_1}**2/2 + I2*(\dot{θ_1} + \dot{θ_2})**2/2 + \dot{θ_1}**2*m1*r1**2/2 - g*m1*r1*cos(θ_1) + g*m2*(r2*cos(θ_1 + θ_2) - r_j*cos(θ_1)) + m2*(\dot{θ_1}**2*r2**2 - 2*\dot{θ_1}**2*r2*r_j*cos(θ_2) + \dot{θ_1}**2*r_j**2 + 2*\dot{θ_1}*\dot{θ_2}*r2**2 - 2*\dot{θ_1}*\dot{θ_2}*r2*r_j*cos(θ_2) + \dot{θ_2}**2*r2**2)/2

In [6]:
#Euler-Lagrange
tau = symbols('tau')
Eq_th1 = clean(Eq(diff(diff(L, diff(th1)), t) - diff(L, th1), 0))
Eq_th2 = clean(Eq(diff(diff(L, diff(th2)), t) - diff(L, th2), tau))

display(Eq_th1, Eq_th2)

Eq(I1*\ddot{θ_1} + I2*(2*\ddot{θ_1} + 2*\ddot{θ_2})/2 + \ddot{θ_1}*m1*r1**2 - g*m1*r1*sin(θ_1) - g*m2*(-r2*sin(θ_1 + θ_2) + r_j*sin(θ_1)) + m2*(2*\ddot{θ_1}*r2**2 - 4*\ddot{θ_1}*r2*r_j*cos(θ_2) + 2*\ddot{θ_1}*r_j**2 + 2*\ddot{θ_2}*r2**2 - 2*\ddot{θ_2}*r2*r_j*cos(θ_2) + 4*\dot{θ_1}*\dot{θ_2}*r2*r_j*sin(θ_2) + 2*\dot{θ_2}**2*r2*r_j*sin(θ_2))/2, 0)

Eq(I2*(2*\ddot{θ_1} + 2*\ddot{θ_2})/2 + g*m2*r2*sin(θ_1 + θ_2) - m2*(2*\dot{θ_1}**2*r2*r_j*sin(θ_2) + 2*\dot{θ_1}*\dot{θ_2}*r2*r_j*sin(θ_2))/2 + m2*(2*\ddot{θ_1}*r2**2 - 2*\ddot{θ_1}*r2*r_j*cos(θ_2) + 2*\ddot{θ_2}*r2**2 + 2*\dot{θ_1}*\dot{θ_2}*r2*r_j*sin(θ_2))/2, tau)

In [7]:
# solve for ddth1 in terms of ddth2 using Eq1
ddth1 = symbols('\\ddot{θ_1}')
ddth2 = symbols('\\ddot{θ_2}')
sol = solve([Eq_th1], ddth1)
ddth1_sol = simplify(sol[ddth1])
ddth1_sol

(-I2*\ddot{θ_2} - \ddot{θ_2}*m2*r2**2 + \ddot{θ_2}*m2*r2*r_j*cos(θ_2) - 2*\dot{θ_1}*\dot{θ_2}*m2*r2*r_j*sin(θ_2) - \dot{θ_2}**2*m2*r2*r_j*sin(θ_2) + g*m1*r1*sin(θ_1) - g*m2*r2*sin(θ_1 + θ_2) + g*m2*r_j*sin(θ_1))/(I1 + I2 + m1*r1**2 + m2*r2**2 - 2*m2*r2*r_j*cos(θ_2) + m2*r_j**2)

In [8]:
# substitute into Eq2, solve for ddth2
Eq_th2 = Eq_th2.subs({ddth1:ddth1_sol})
sol = solve([Eq_th2], ddth2)
ddth2_sol = simplify(sol[ddth2])
ddth2_sol

(I1*\dot{θ_1}**2*m2*r2*r_j*sin(θ_2) - I1*g*m2*r2*sin(θ_1 + θ_2) + I1*tau + I2*\dot{θ_1}**2*m2*r2*r_j*sin(θ_2) + 2*I2*\dot{θ_1}*\dot{θ_2}*m2*r2*r_j*sin(θ_2) + I2*\dot{θ_2}**2*m2*r2*r_j*sin(θ_2) - I2*g*m1*r1*sin(θ_1) - I2*g*m2*r_j*sin(θ_1) + I2*tau + \dot{θ_1}**2*m1*m2*r1**2*r2*r_j*sin(θ_2) + \dot{θ_1}**2*m2**2*r2**3*r_j*sin(θ_2) - \dot{θ_1}**2*m2**2*r2**2*r_j**2*sin(2*θ_2) + \dot{θ_1}**2*m2**2*r2*r_j**3*sin(θ_2) + 2*\dot{θ_1}*\dot{θ_2}*m2**2*r2**3*r_j*sin(θ_2) - \dot{θ_1}*\dot{θ_2}*m2**2*r2**2*r_j**2*sin(2*θ_2) + \dot{θ_2}**2*m2**2*r2**3*r_j*sin(θ_2) - \dot{θ_2}**2*m2**2*r2**2*r_j**2*sin(2*θ_2)/2 - g*m1*m2*r1**2*r2*sin(θ_1 + θ_2) - g*m1*m2*r1*r2**2*sin(θ_1) + g*m1*m2*r1*r2*r_j*sin(θ_1 - θ_2)/2 + g*m1*m2*r1*r2*r_j*sin(θ_1 + θ_2)/2 - g*m2**2*r2**2*r_j*sin(θ_1)/2 + g*m2**2*r2**2*r_j*sin(θ_1 + 2*θ_2)/2 + g*m2**2*r2*r_j**2*sin(θ_1 - θ_2)/2 - g*m2**2*r2*r_j**2*sin(θ_1 + θ_2)/2 + m1*r1**2*tau + m2*r2**2*tau - 2*m2*r2*r_j*tau*cos(θ_2) + m2*r_j**2*tau)/(I1*I2 + I1*m2*r2**2 + I2*m1*r1**2 + I2*m2*

In [9]:
# substitute back into Eq1, solve for ddth1
Eq_th1 = Eq_th1.subs({ddth2:simplify(ddth2_sol)})
sol = solve([Eq_th1], ddth1)
ddth1_sol = simplify(sol[ddth1])
ddth1_sol

(-I1*I2*\dot{θ_1}**2*m2*r2*r_j*sin(θ_2) - 2*I1*I2*\dot{θ_1}*\dot{θ_2}*m2*r2*r_j*sin(θ_2) - I1*I2*\dot{θ_2}**2*m2*r2*r_j*sin(θ_2) + I1*I2*g*m1*r1*sin(θ_1) + I1*I2*g*m2*r_j*sin(θ_1) - I1*I2*tau - I1*\dot{θ_1}**2*m2**2*r2**3*r_j*sin(θ_2) + I1*\dot{θ_1}**2*m2**2*r2**2*r_j**2*sin(2*θ_2)/2 - 2*I1*\dot{θ_1}*\dot{θ_2}*m2**2*r2**3*r_j*sin(θ_2) - I1*\dot{θ_2}**2*m2**2*r2**3*r_j*sin(θ_2) + I1*g*m1*m2*r1*r2**2*sin(θ_1) + I1*g*m2**2*r2**2*r_j*sin(θ_1)/2 - I1*g*m2**2*r2**2*r_j*sin(θ_1 + 2*θ_2)/2 - I1*m2*r2**2*tau + I1*m2*r2*r_j*tau*cos(θ_2) - I2**2*\dot{θ_1}**2*m2*r2*r_j*sin(θ_2) - 2*I2**2*\dot{θ_1}*\dot{θ_2}*m2*r2*r_j*sin(θ_2) - I2**2*\dot{θ_2}**2*m2*r2*r_j*sin(θ_2) + I2**2*g*m1*r1*sin(θ_1) + I2**2*g*m2*r_j*sin(θ_1) - I2**2*tau - I2*\dot{θ_1}**2*m1*m2*r1**2*r2*r_j*sin(θ_2) - 2*I2*\dot{θ_1}**2*m2**2*r2**3*r_j*sin(θ_2) + 3*I2*\dot{θ_1}**2*m2**2*r2**2*r_j**2*sin(2*θ_2)/2 - I2*\dot{θ_1}**2*m2**2*r2*r_j**3*sin(θ_2) - 2*I2*\dot{θ_1}*\dot{θ_2}*m1*m2*r1**2*r2*r_j*sin(θ_2) - 4*I2*\dot{θ_1}*\dot{θ_2}*m2**2*r

In [10]:
from sympy.codegen.ast import CodeBlock, Assignment

ddth1_num, ddth2_num = symbols('ddth1_num, ddth2_num')

codeblock = CodeBlock(
    Assignment(ddth1_num, ddth1_sol),
    Assignment(ddth2_num, ddth2_sol),
).cse() #common subexpression elimination
num_ops = count_ops(codeblock)
code = pycode(codeblock)

replace_dict = {
    '\dot': 'd',
    '\ddot': 'dd',
    'θ_1': 'theta1',
    'θ_2': 'theta2',
}
for item in replace_dict:
    code = code.replace(item, replace_dict[item])

print(f"# {num_ops} operations")
print(code)

# 352 operations
x0 = I2**2
x1 = I1**2
x2 = m2**2
x3 = r2**4
x4 = x2*x3
x5 = r2**2
x6 = m2*x5
x7 = m1**2
x8 = r1**4*x7
x9 = r_j**4
x10 = I2*x2
x11 = r1**2
x12 = m1*x11
x13 = r_j**2
x14 = m2*x13
x15 = I1*I2
x16 = 2*x15
x17 = m2*r_j
x18 = math.cos(theta2)
x19 = r2*x18
x20 = x13*x5
x21 = I1*x2
x22 = x20*x21
x23 = x10*x20
x24 = I1*x6
x25 = 2*x12
x26 = I2*x12
x27 = 2*x6
x28 = 2*x26
x29 = 2*x18
x30 = r2**3
x31 = r_j*x30
x32 = x21*x31
x33 = r_j**3
x34 = x10*x33
x35 = 2*x19
x36 = m2**3
x37 = math.sin(theta2)
x38 = x37**2
x39 = x36*x38
x40 = x2*x20
x41 = x2*x31
x42 = x30*x33
x43 = x38*x40
x44 = I1*tau
x45 = I2*tau
x46 = math.sin(theta1)
x47 = tau*x12
x48 = dtheta1**2
x49 = x37*x48
x50 = r2*x49
x51 = x17*x50
x52 = dtheta2**2
x53 = x37*x52
x54 = r2*x53
x55 = x17*x54
x56 = dtheta1*dtheta2
x57 = x37*x56
x58 = r2*x57
x59 = x17*x58
x60 = tau*x40
x61 = r1**3
x62 = x0*x17
x63 = 2*x37*x56
x64 = r2*x63
x65 = math.sin(theta1 + theta2)
x66 = g*r2
x67 = x65*x66
x68 = m1*r1
x69 = x17*x68
x70 = I2*x69
x71 = r

In [11]:
J = Matrix([
    [x1.diff(th1), x1.diff(th2)],
    [y1.diff(th1), y1.diff(th2)],
    [x2.diff(th2), x2.diff(th2)],
    [y2.diff(th2), y2.diff(th2)],
])
q = Matrix([th1, th2])
dq = q.diff(t)
v = simplify(J*dq)
a = simplify(J*dq.diff(t) + J.diff(t)*dq)

display(
    Eq(MatrixSymbol('J', 4, 2), clean(J)), 
    Eq(MatrixSymbol('q', 2, 1), clean(q)),
    Eq(MatrixSymbol('dq', 2, 1), clean(dq)),
    Eq(MatrixSymbol('v', 4, 1), clean(v)),
    Eq(MatrixSymbol('a', 4, 1), clean(a))
)

Eq(J, Matrix([
[     -r1*cos(θ_1),                 0],
[     -r1*sin(θ_1),                 0],
[r2*cos(θ_1 + θ_2), r2*cos(θ_1 + θ_2)],
[r2*sin(θ_1 + θ_2), r2*sin(θ_1 + θ_2)]]))

Eq(q, Matrix([
[θ_1],
[θ_2]]))

Eq(dq, Matrix([
[\dot{θ_1}],
[\dot{θ_2}]]))

Eq(v, Matrix([
[                   -\dot{θ_1}*r1*cos(θ_1)],
[                   -\dot{θ_1}*r1*sin(θ_1)],
[r2*(\dot{θ_1} + \dot{θ_2})*cos(θ_1 + θ_2)],
[r2*(\dot{θ_1} + \dot{θ_2})*sin(θ_1 + θ_2)]]))

Eq(a, Matrix([
[                                                                                                               r1*(-\ddot{θ_1}*cos(θ_1) + \dot{θ_1}**2*sin(θ_1))],
[                                                                                                               -r1*(\ddot{θ_1}*sin(θ_1) + \dot{θ_1}**2*cos(θ_1))],
[r2*(\ddot{θ_1}*cos(θ_1 + θ_2) + \ddot{θ_2}*cos(θ_1 + θ_2) - \dot{θ_1}*(\dot{θ_1} + \dot{θ_2})*sin(θ_1 + θ_2) - \dot{θ_2}*(\dot{θ_1} + \dot{θ_2})*sin(θ_1 + θ_2))],
[r2*(\ddot{θ_1}*sin(θ_1 + θ_2) + \ddot{θ_2}*sin(θ_1 + θ_2) + \dot{θ_1}*(\dot{θ_1} + \dot{θ_2})*cos(θ_1 + θ_2) + \dot{θ_2}*(\dot{θ_1} + \dot{θ_2})*cos(θ_1 + θ_2))]]))

In [12]:
a_1y, a_2y = symbols('a_1y, a_2y')

codeblock = CodeBlock(
    Assignment(a_1y, clean(a[1])),
    Assignment(a_2y, clean(a[3])),
).cse() #common subexpression elimination
num_ops = count_ops(codeblock)
code = pycode(codeblock)

for item in replace_dict:
    code = code.replace(item, replace_dict[item])

print(f"# {num_ops} operations")
print(code)

# 23 operations
x0 = theta1 + theta2
x1 = math.sin(x0)
x2 = (dtheta1 + dtheta2)*math.cos(x0)
a_1y = -r1*(ddtheta1*math.sin(theta1) + dtheta1**2*math.cos(theta1))
a_2y = r2*(ddtheta1*x1 + ddtheta2*x1 + dtheta1*x2 + dtheta2*x2)
